<div style="text-align:center">
    <h1> Functions </h1>
    <h2> CS3100 Monsoon 2020 </h2>
</div>

## Review

Previously on CS3100

* Syntax and Semantics
* Expressions: if, let
* Definitions: let

Today

* Functions

## Anonymous Function

OCaml has support for anonymous function expressions. The syntax is

```ocaml
fun x1 ... xn -> e
```

* A function is a value; no further computation to do.
* In particular, `e` is not evaluated until the function is applied.

## Anonymous Functions

In [ ]:
fun x -> x +. 1.

The function type `int -> int` says that it takes one argument of type `int` and returns a value of type `int`.

## Thunk

In [ ]:
fun () -> 1

* A function of type `unit -> t` is called a `thunk`. 
* Delay the computation of the RHS expression until application.

## Anonymous Functions

The function body can refer to any variables in scope. 

In [ ]:
let foo = 
  let y = 10 in
  let x = 5 in
  fun z -> x + y + z

The anonymous function `fun z -> x + y + z` is said to **_close over_** the **_free variables_** `x` and `y`. 

## Quiz

What is the value of `res`?

In [ ]:
let bar =
  let x = 5 in
  fun y -> x + y
  
let res = bar 10

## Functions are values

Can use them *anywhere* we can use values:

* Functions can **take** functions as arguments
* Functions can **return** functions as arguments

As you will see, this is an incredibly powerful language feature. 

## Function application

The syntax is 

```ocaml
e0 e1 ... en
```
* No parentheses necessary

## Function Application Evaluation

```ocaml
e0 e1 ... en
```

* Evaluate `e0 ... en` to values `v0 ... vn`
* Type checking will ensure that `v0` is a function `fun x1 ... xn -> e`
* Substitute `vi` for `xi` in `e` yielding new expression `e'`
* Evaluate `e'` to a value `v`, which is result

## Function Application

In [ ]:
(fun x -> x + 1) 1

In [ ]:
(fun x y z -> x + y + z) 1 2 3

## Function Application is left associative

Read

```ocaml
(fun x y z -> x + y + z) 1 2 3
```

as 

```ocaml
((((fun x y z -> x + y + z) 1) 2) 3)
```

In [ ]:
((((fun x y z -> x + y + z) 1) 2) 3)

## Multi-argument functions are syntactic sugar

The function

```ocaml
(fun x y z -> x + y + z) 1 2 3
```

In [ ]:
(fun x -> fun y -> fun z -> x + y + z) 1 2 3

## Function definition

We can name functions using `let`.

```ocaml
let succ = fun x -> x + 1
```

which is semantically equivalent to

```ocaml
let succ x = x + 1
```

You'll see the latter form more often.

## Function definition

In [ ]:
let succ x = x + 1

In [ ]:
succ 10

## Function definition

In [ ]:
let add x y = x + y

In [ ]:
let add = fun x -> fun y -> x + y

In [ ]:
add 5 10

## Function types are right associative

```ocaml
int -> int -> int
```

is equivalent to

```ocaml
int -> (int -> int)
```

In [ ]:
let add : int -> (int -> int) = 
  fun x -> fun y -> x + y

This reading is quite useful!

## Partial Application

A function of type

```ocaml
int -> int -> int -> int
```

can be read as 

```ocaml
int -> (int -> int -> int)
```

Take 1 argument and return a function!

## Partial Application

```ocaml
(fun x y z -> x + y + z) 1
```

returns a function 

```ocaml
(fun y z -> 1 + y + z)
```

In [ ]:
let foo = (fun x y z -> x + y + z) 1

In [ ]:
foo 2 3

## Partial Application

A more useful partial application example is defining `succ` and `pred` functions from `add`.

In [ ]:
let succ = add 1
let pred = add (-1)

In [ ]:
succ 10

In [ ]:
pred 10

## Recursive Functions

Recursive functions can call themselves. The syntax for recursive function definition is:

```ocaml
let rec foo x = ...
```

Notice the `rec` key word.

## Recursive Functions

In [ ]:
let rec sum_of_first_n n = 
  if n <= 0 then 0
  else n + sum_of_first_n (n-1)

In [ ]:
sum_of_first_n 5

## Tracing functions in Jupyter

Jupyter (really the ocaml top-level behind the scenes) provides support for tracing the execution of functions.

In [ ]:
#trace sum_of_first_n;;

sum_of_first_n 3;;

In [ ]:
#untrace sum_of_first_n

## Recursive Fibonacci number

Implement a recursive function that computes the nth fibonacci number. The fibonacci sequence is `[0;1;1;2;3;5;8;...]`.

In [ ]:
let rec fib n = 
  if n = 0 then 0
  else if n = 1 then 1
  else fib (n - 2) + fib (n - 1)

In [ ]:
assert (fib 10 = 55)

In OCaml, the expression `assert e` raises a fatal error if `e` evaluates to `false`. If `e` is `true`, then `assert e` evaluates to `true`.

## Mutually recursive functions

Mutually recursive functions are a set of functions that can call each other. They are defined using

```
let rec f1 v1 v2 ... vn =
  ...
and f2 v1 v2 ... vn =
   ...
...
and fn v1 v2 ... vn =
   ...
```

where the functions `f1` to `fn` can call each other. 

In [ ]:
let rec even n =
  if n = 0 then true
  else odd (n-1)
  
and odd n = 
  if n = 0 then false 
  else even (n-1)

## Recursing too deep

Let's invoke `sum_of_first_n` with larger numbers. 

In [ ]:
sum_of_first_n 1000000

## Stack buildup

```ocaml
let rec sum_of_first_n n = 
  if n <= 0 then 0
  else n + sum_of_first_n (n-1)
```

Some work `"+ n"` left to do after the recursive call returns. This builds up stack frames.

## Stack buildup

For `sum_of_first_n 5`:

<center>

<img src="images/stack1.svg" width="150">
</center>

## Stack buildup

For `sum_of_first_n 5`:

<center>

<img src="images/stack2.svg" width="225">
</center>

## Tail recursion

Rewrite the function such that the recursive call is the last thing that the function does:

In [ ]:
let rec sum_of_first_n_tailrec acc n = 
  if n <= 0 then acc
  else sum_of_first_n_tailrec (n + acc) (n-1)
  
let sum_of_first_n_tailrec n = sum_of_first_n_tailrec 0 n

In [ ]:
sum_of_first_n_tailrec 1000000000

## Tail recursion

```ocaml
let rec sum_of_first_n_tailrec n acc = 
  if n <= 0 then acc
  else sum_of_first_n_tailrec (n-1) (n + acc)
```

* No work left to do when the recursive call returns except return result to caller.
* OCaml compiler does **tail call optimisation** that pops current call frame before invoking recursive call. 
  + No stack buildup => equivalent to writing a tight loop. 

<div style="text-align:center">
    <h1> Fin. </h1>
</div>